In [5]:
import sys
sys.path.append('amrprediction/')
from Bio.Phylo import BaseTree
from Bio import Phylo
from amrprediction.isolate_relation_analysis import IsolatesRelations
import itertools
import numpy as np
import codecs

In [6]:
def get_path_edges(node_seq):
    '''
    from node sequence to edges
    '''
    edges=list()
    for first, second in zip(node_seq, node_seq[1:]):
        edges.append(([x.name for x in first.get_terminals()],[x.name for x in second.get_terminals()]))
    return edges

In [7]:
def gain_loss_edges(ISR, A, B, drug_idx):
    isolates=ISR.DrugRel.get_isolate_list()
    idxs_A=[isolates.index(x) for x in A if x in isolates]
    idxs_B=[isolates.index(x) for x in B if x in isolates]
    A_labels=[ISR.DrugRel.drug_vectors[drug_idx,idx] for idx in idxs_A]
    B_labels=[ISR.DrugRel.drug_vectors[drug_idx,idx] for idx in idxs_B]
    A_labels=list(set(A_labels))
    B_labels=list(set(B_labels))
    A_labels.sort()
    B_labels.sort()
    check={str([0.5]):'I',str([0.0,1.0]):'SR',str([]):'NONE', str([1.0]):'R',str([0.0]):'S',str([0.5,1.0]):'IR',str([0.0,0.5]):'SI',str([0.0,0.5,1.0]):'SIR'}
    result=(check[str(A_labels)],check[str(B_labels)])

    final_check={('SIR','I'):'I',('SIR','R'):'R',('SIR','S'):'S',('SI','I'):'I',('SI','S'):'S',('IR','I'):'I',('IR','R'):'R',('SR','R'):'R',('SR','S'):'S'}    
    if result in final_check:
        return A_labels, B_labels, final_check[result]
    else:
        return False

In [8]:
tree = Phylo.read("amrprediction/data_config/tree.nwk", "newick")
ISR=IsolatesRelations()
terminals=tree.get_terminals()
all_edges=list()
for t in terminals:
    all_edges.append(get_path_edges(tree.get_path(t)))
all_edges_tot=list(itertools.chain(*all_edges))

In [ ]:
# check results
for edge in all_edges_tot:
    res=gain_loss_edges(ISR, edge[0], edge[1],0)
    if res:
        print(gain_loss_edges(ISR, edge[0], edge[1],0))

In [17]:
import codecs
for drug_idx, drug in enumerate(ISR.DrugRel.drugs):
    print(drug)
    f=codecs.open('amrprediction/data_config/GL_'+drug+'.txt','w','utf-8')
    for edge in all_edges_tot:
        res=gain_loss_edges(ISR, edge[0], edge[1],drug_idx)
        if res:
            f.write('\t'.join([' '.join(edge[0]),' '.join(edge[1]), '\t'.join([str(r) for r in res])])+'\n')
    f.close()

Ciprofloxacin
Tobramycin
Colistin
Ceftazidim
Meropenem


## Create mapping

In [18]:
isolates=ISR.DrugRel.get_isolate_list()
idxs_A=[isolates.index(x) for x in isolates]
label_map={0:'V',1:'R',0.5:'I'}
maps=list()
for drugidx,drug in enumerate(ISR.DrugRel.drugs):
    mapping={isol:label_map[ISR.DrugRel.drug_vectors[drugidx,idx]] for idx, isol in enumerate(isolates)}
    maps.append(mapping)

